In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/projet_traffic_sign_detection_RCNN_Faster
!ls

/content/drive/MyDrive/projet_traffic_sign_detection_RCNN_Faster
data  Untitled0.ipynb


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


Install biblioteque

In [ ]:
! pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

In [ ]:
import torch
assert torch.__version__.startswith("1.8")
import torchvision
import cv2

In [ ]:
torch.cuda.is_available()


In [ ]:
import os
import cv2
import torch
import detectron2
import matplotlib.pyplot as plt

from detectron2.structures import BoxMode
from detectron2.data import DatasetCataIog, MetadataCataIog

In [5]:
pwd

'/content/drive/MyDrive/projet_traffic_sign_detection_RCNN_Faster'

In [ ]:

def load_class_mapping(csv_path):
    df = pd.read_csv(csv_path)
    class_map = dict(zip(df["SignName"], df["ClassId"]))
    return class_map

In [ ]:
def load_dataset_from_csv(csv_annotations, images_dir, class_map):
    df = pd.read_csv(csv_annotations)

    dataset_dicts = []
    image_id = 0

    for file_name, group in df.groupby("file_name"):
        record = {}

        record["file_name"] = os.path.join(images_dir, file_name)
        record["image_id"] = image_id
        record["height"] = int(group.iloc[0]["height"])
        record["width"] = int(group.iloc[0]["width"])

        image_id += 1

        objs = []
        for _, row in group.iterrows():
            label = row["class_name"]

            if label not in class_map:
                print(f"⚠️ Class not found: {label}")
                continue

            obj = {
                "bbox": [
                    float(row["x_min"]),
                    float(row["y_min"]),
                    float(row["x_max"]),
                    float(row["y_max"]),
                ],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": class_map[label],
                "iscrowd": 0,
            }
            objs.append(obj)

        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts


In [ ]:

class_map = load_class_mapping("/content/drive/MyDrive/projet_traffic_sign_detection_RCNN_Faster/data/labels/sign_names.csv")

for d in ["train", "test"]:
    DatasetCatalog.register(
        f"traffic_signs_{d}",
        lambda d=d: load_dataset_from_csv(
            csv_annotations=f"/content/drive/MyDrive/projet_traffic_sign_detection_RCNN_Faster/data/annotations/{d}_annotation.csv",
            images_dir=f"/content/drive/MyDrive/projet_traffic_sign_detection_RCNN_Faster/data/images/{d}",
            class_map=class_map
        )
    )

    MetadataCatalog.get(f"traffic_signs_{d}").set(
        thing_classes=list(class_map.keys())
    )


In [ ]:
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer,ColorMode
from detectron2.data import MetadataCatalog
from google.colab.patches import cv2_imshow

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))

# Datasets
cfg.DATASETS.TRAIN = ("my_train",)
cfg.DATASETS.TEST = ()

# Dataloader
cfg.DATALOADER.NUM_WORKERS = 2

# Model weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")

# Solver / training hyperparameters
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.STEPS = [500]
cfg.SOLVER.MAX_ITER = 2000

# ROI heads / number of classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 43

# Device
cfg.MODEL.DEVICE = "cuda"

In [ ]:
cfg.OUTPUT_DIR = "./output_faster_rcnn"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
! pip install tensorboard

In [ ]:
import tensorboard
%load_ext tensorboard
%tensorboard --logdir output_faster_rcnn


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # threshhold for detection
cfg.DATASETS.TEST = ("my_test" )
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_test")
test_dataset_dicts = get_data_dicts(data_path+'test', classes)

In [ ]:

for d in random.sample(test_dataset_dicts, 4):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    v = Visualizer(
        img[:, :, ::-1],   # BGR → RGB
        metadata=microcontroller_metadata,
        scale=0.8
    )

    out = v.draw_instance_predictions(
        outputs["instances"].to("cpu")
    )

    plt.figure(figsize=(14, 8))
    plt.imshow(out.get_image())
    plt.axis("off")
    plt.show()
